### Running Palace Simulations on GDSFactory+ Cloud


In [ ]:
!uv pip install "gsim @ git+https://github.com/gdsfactory/gsim.git"

In [ ]:
import gsim.palace as gpalace

In [ ]:
# === Configuration ===
OUTPUT_DIR = "./palace-sim-microstrip"

### Load a pcell from IHP PDK

In [ ]:
import gdsfactory as gf

from ihp import LAYER, PDK, cells

PDK.activate()

c = gf.Component()
r1 = c << cells.straight_metal()

r = c.get_region(layer=LAYER.TopMetal2drawing)
r_sized = r.sized(+5000)
c.add_polygon(r_sized, layer=LAYER.Metal1drawing)


c.add_ports(r1.ports)

cc = c.copy()
cc.draw_ports()
cc

### Generate the mesh

In [ ]:
# Get stack (no substrate for RF simulation)
stack = gpalace.get_stack(substrate_thickness=2.0, air_above=300.0)

# Configure via ports (Metal1 to TopMetal2)
gpalace.configure_via_port(
    c.ports,
    from_layer="metal1",
    to_layer="topmetal2",
    impedance=50.0,
)

ports = gpalace.extract_ports(c, stack)

# Generate mesh
result = gpalace.generate_mesh(
    component=c,
    stack=stack,
    ports=ports,
    output_dir=OUTPUT_DIR,
)

In [ ]:
# Static PNG
gpalace.plot_mesh(
    f"{OUTPUT_DIR}/palace.msh",
    output=f"{OUTPUT_DIR}/mesh.png",
    show_groups=["metal", "P"],
    interactive=False,
)

# Interactive
# gpalace.plot_mesh(f"{OUTPUT_DIR}/palace.msh", show_groups=["metal", "P"], interactive=True)

### Run simulation on GDSFactory+ Cloud

In [ ]:
results = gpalace.run_simulation(OUTPUT_DIR)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(results["port-S.csv"])
df.columns = df.columns.str.strip()  # Remove whitespace from column names

freq = df["f (GHz)"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

# Magnitude plot
ax1.plot(freq, df["|S[1][1]| (dB)"], marker=".", label="S11")
ax1.plot(freq, df["|S[2][1]| (dB)"], marker=".", label="S21")
ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("Magnitude (dB)")
ax1.set_title("S-Parameters")
ax1.legend()
ax1.grid(True)

# Phase plot
ax2.plot(freq, df["arg(S[1][1]) (deg.)"], marker=".", label="S11")
ax2.plot(freq, df["arg(S[2][1]) (deg.)"], marker=".", label="S21")
ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("Phase (deg)")
ax2.legend()
ax2.grid(True)

plt.tight_layout()